In [9]:
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, AveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.activations import relu, sigmoid, softmax
from tensorflow.keras import backend as K
import sys
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
from matplotlib import colors
%matplotlib notebook

# Helper Functions

## create_random_input

Creates a random input vector in the desired shape. The numbers are between 0 and size / 128.

**Input Parameters**
* shape -- The desired shape
* divider -- The divider to scale the output by

**Output Parameters**
* The desired tensor

## ref_conf2d

The reference Conv2D implementation from TensorFlow. 

**Input Parameters**
* X -- The input tensor
* w -- The weights
* b -- The bias
* kernel_size -- The kernel size (kernel_rows, kernel_cols)
* strides -- The strides (stride_rows, stride_cols)
* padding -- "valid" for no padding, "same" for zero padding
* groups -- The number of desired groups

**Output Parameters**
* The result tensor of the convolution

## check_conv2d

Checks an input function with the given parameters against the reference implementation of TensorFlow. 

**Input Parameters**
* fn -- The function to compare to
* shape -- The shape to test the function with
* kernel_size -- The kernel size (kernel_rows, kernel_cols) 
* filters -- The number of filters to use
* strides -- The strides (stride_rows, stride_cols)
* padding -- "valid" for no padding, "same" for zero padding
* groups -- The number of desired groups

**Output Parameters**
* True if the function produces the same output as the reference, False otherwise

## Important Note

Due to floating point inaccuracies (hopefully), the results are not equal but close instead. 

In [50]:
def create_random_input(shape, divider=128.0):
    num_elements = np.prod(shape)
    return np.random.randint(0, num_elements, num_elements).astype(np.float32).reshape(shape) / divider

def ref_conv2d(X, w, b, kernel_size, strides, padding, groups):
    return Conv2D(input_shape=X.shape, weights=[w, b], kernel_size=kernel_size, filters=w.shape[3], strides=strides, padding=padding, groups=groups)(X).numpy()
    
def check_conv2d(fn, shape, kernel_size, filters, strides, padding, groups=1):
    np.random.seed(3)
    X = create_random_input(shape)
    w = create_random_input((kernel_size[0], kernel_size[1], shape[3], filters))
    b = create_random_input((filters,))
    
    ref = ref_conv2d(X, w, b, kernel_size, strides, padding, groups)
    res = fn(X, w, b, kernel_size, strides, padding, groups)
    
    return np.allclose(res, ref)

# Conv2D Implementation

The actual Conv2D implementation. This function has the same input and output parameters as the ref_conv2d function. 

In [12]:
def res_conv2d(X, w, b, kernel_size, strides, padding, groups=1):
    # extract parameters
    (kernel_rows, kernel_cols, channels_in, channels_out) = w.shape
    (batch, rows_in, cols_in, channels_in_) = X.shape
    grouped_channels_out = channels_out//groups
    
    # check parameter compatibility
    assert channels_in * groups == channels_in_
    assert channels_out % groups == 0
    
    # same padding
    if padding == "same":
        rows_offset = kernel_rows//2
        cols_offset = kernel_cols//2
    
        # calculate output dimensions
        rows_out = (rows_in + strides[0] - 1) // strides[0]
        cols_out = (cols_in + strides[1] - 1) // strides[1]
        
    
        # create output buffer
        out = np.zeros((int(batch), int(rows_out), int(cols_out), int(groups*channels_out)))

        # prefill the output with bias
        for i in range(batch):
            for y in range(rows_out):
                for x in range(cols_out):
                    for g in range(groups):
                        gc = g * grouped_channels_out
                        for co in range(grouped_channels_out):
                            out[i, y, x, gc + co] = b[co]

        # convolute
        for i in range(batch):
            for y in range(rows_out):
                for x in range(cols_out):
                    sy = y * strides[0] - rows_offset
                    sx = x * strides[1] - rows_offset
                    for g in range(groups):
                        for co in range(grouped_channels_out):
                            for ci in range(channels_in):
                                for ky in range(sy, sy + kernel_rows):
                                    for kx in range(sx, sx + kernel_cols):
                                        if ky >= 0 and ky < rows_in and kx >= 0 and kx < cols_in:
                                            gc = g * grouped_channels_out
                                            out[i, y, x, gc + co] += X[i, ky , kx, gc + ci] * w[(ky - sy), (kx - sx), ci, co]

        return out
    
    # zero padding
    else:
        rows_out = (rows_in - (kernel_rows - strides[0])) // strides[0]
        cols_out = (cols_in - (kernel_cols - strides[1])) // strides[1]
        
    
        # create output buffer
        out = np.zeros((int(batch), int(rows_out), int(cols_out), int(groups*channels_out)))

        # prefill the output with bias
        for i in range(batch):
            for y in range(rows_out):
                for x in range(cols_out):
                    for g in range(groups):
                        gc = g * grouped_channels_out
                        for co in range(grouped_channels_out):
                            out[i, y, x, gc + co] = b[co]

        # convolute
        for i in range(batch):
            for y in range(rows_out):
                for x in range(cols_out):
                    sy = y * strides[0]
                    sx = x * strides[1]
                    for g in range(groups):
                        for co in range(grouped_channels_out):
                            for ci in range(channels_in):
                                for ky in range(sy, sy + kernel_rows):
                                    for kx in range(sx, sx + kernel_cols):
                                        gc = g * grouped_channels_out
                                        out[i, y, x, gc + co] += X[i, ky , kx, gc + ci] * w[(ky - sy), (kx - sx), ci, co]

        return out

print(check_conv2d(res_conv2d, (2, 128, 128, 4), (3, 3), 5, (1, 1), "same"))
#print(check_conv2d(res_conv2d, (1, 3, 3, 1), (3, 3), 1, (1, 1), "valid"))
#print(check_conv2d(res_conv2d, (1, 4, 4, 1), (1, 1), 1, (1, 1), "valid"))

True


# More Helper Functions

## ref_dense

The reference Dense implementation from TensorFlow. 

**Input Parameters**
* X -- The input tensor
* w -- The weights
* b -- The bias

**Output Parameters**
* The result tensor of the convolution

## check_dense

Checks an input function with the given parameters against the reference implementation of TensorFlow. 

**Input Parameters**
* fn -- The function to compare to
* units -- The dimension of the output
* shape -- The shape to test the function with

**Output Parameters**
* True if the function produces the same output as the reference, False otherwise

## Important Note

Due to floating point inaccuracies (hopefully), the results are not equal but close instead. 

In [42]:
def ref_dense(X, w, b):
    return Dense(w.shape[1], input_shape=X.shape, weights=[w, b])(X).numpy()
    
def check_dense(fn, units, shape):
    np.random.seed(3)
    shape = np.array(shape)
    size = np.prod(shape[1:])
    
    X = create_random_input((shape[0], size))
    w = create_random_input((size, units))
    b = create_random_input((units))
    
    ref = ref_dense(X, w, b)
    res = fn(X, w, b)
    
    #return (res == ref).all()
    return np.allclose(res, ref)

# Dense Layer Implementation

The actual dense layer implementation. This function has the same input and output parameters as the ref_dense function. 

In [43]:
def res_dense(X, w, b):
    # extract parameters
    (batches, size) = X.shape
    (w_s, units) = w.shape
    
    # check parameter compatibility
    assert size == w_s
    
    # create output buffer
    out = np.zeros((batches, units))
        
    # prefill output with bias
    for i in range(batches):
        for j in range(units):
            out[i, j] = b[j]
    
    # matrix multiplikation
    for i in range(batches):
        for j in range(units):
            for k in range(size):
                out[i, j] += X[i, k] * w[k, j]
                
    return out

print(check_dense(res_dense, 3, (5, 256)))

True


# Even More Helper Functions

## ref_relu

The reference ReLU implementation from TensorFlow. 

**Input Parameters**
* X -- The input tensor


**Output Parameters**
* The result tensor of the convolution

## check_relu

Checks an input function with the given parameters against the reference implementation of TensorFlow. 

**Input Parameters**
* fn -- The function to compare to
* shape -- The shape to test the function with

**Output Parameters**
* True if the function produces the same output as the reference, False otherwise

## ref_sigmoid

The reference sigmoid implementation from TensorFlow. 

**Input Parameters**
* X -- The input tensor


**Output Parameters**
* The result tensor of the convolution

## check_sigmoid

Checks an input function with the given parameters against the reference implementation of TensorFlow. 

**Input Parameters**
* fn -- The function to compare to
* shape -- The shape to test the function with

**Output Parameters**
* True if the function produces the same output as the reference, False otherwise

## ref_softmax

The reference softmax implementation from TensorFlow. 

**Input Parameters**
* X -- The input tensor


**Output Parameters**
* The result tensor of the convolution

## check_softmax

Checks an input function with the given parameters against the reference implementation of TensorFlow. 

**Input Parameters**
* fn -- The function to compare to
* shape -- The shape to test the function with

**Output Parameters**
* True if the function produces the same output as the reference, False otherwise


## Important Note

Due to floating point inaccuracies (hopefully), the results of the sigmoid functions are not equal but close instead. 

In [79]:
def ref_relu(X):
    return relu(X).numpy()
    
def check_relu(fn, shape):
    np.random.seed(3)
    shape = np.array(shape)
    
    X = create_random_input(shape)
    
    ref = ref_relu(X)
    res = fn(X)
    
    return (res == ref).all()
    #return np.allclose(res, ref)
    
def ref_sigmoid(X):
    return sigmoid(X).numpy()
    
def check_sigmoid(fn, shape):
    np.random.seed(3)
    shape = np.array(shape)
    
    X = create_random_input(shape)
    
    ref = ref_sigmoid(X)
    res = fn(X)
    
    #return (res == ref).all()
    return np.allclose(res, ref)
    
def ref_softmax(X):
    # softmax(K.constant(X)).numpy()
    return softmax(K.constant(X)).numpy()
    
def check_softmax(fn, shape):
    np.random.seed(3)
    shape = np.array(shape)
    
    X = create_random_input(shape, np.prod(shape))
    
    ref = ref_softmax(X)
    res = fn(X)
    
    #return (res == ref).all()
    return np.allclose(res, ref)

# Activation Layer Implementation

The actual relu, sigmoid and softmax activation layer implementations. These function have the same input and output parameters as the ref_relu, ref_sigmoid and ref_softmax functions. 

In [80]:
def res_relu(X):
    # extract parameters
    shape = X.shape
    X = X.flatten()
    
    # create output buffer
    out = np.zeros(X.shape)
    
    # activate all cells
    for i in range(X.shape[0]):
        out[i] = 0 if X[i] <= 0. else X[i]
        
    return out.reshape(shape)

def res_sigmoid(X):
    # extract parameters
    shape = X.shape
    X = X.flatten()
    e = 2.7182818284590452353602874
    
    # create output buffer
    out = np.zeros(X.shape)
    
    # activate all cells
    for i in range(X.shape[0]):
        out[i] = 1 / (1 + e**(-X[i]))
        
    return out.reshape(shape)

def res_softmax(X):
    # extract parameters
    (batches, cells) = X.shape
    e = 2.7182818284590452353602874
    
    # create output buffer
    out = np.zeros(X.shape)
    
    # sum of exponents
    expsums = np.zeros(batches)
    for i in range(batches):
        for j in range(cells):
            expsums[i] += e**X[i, j]
    
    # activate all cells
    for i in range(batches):
        for j in range(cells):
            out[i, j] = e**X[i, j] / expsums[i]
        
    return out

print(check_relu(res_relu, (5, 256)))
print(check_sigmoid(res_sigmoid, (5, 256)))
print(check_softmax(res_softmax, (100, 256)))

True
True
True


# Still Even More Helper Functions

## ref_maxpool2d

The reference max pooling implementation from TensorFlow. 

**Input Parameters**
* X -- The input tensor
* pool_size -- The size of the pool (pool_rows, pool_cols)
* strides -- The strides (stride_rows, stride_cols)
* padding -- "valid" for no padding, "same" for zero padding

**Output Parameters**
* The result tensor of the pooling

## check_maxpool2d

Checks an input function with the given parameters against the reference implementation of TensorFlow. 

**Input Parameters**
* fn -- The function to compare to
* shape -- The shape to test the function with

**Output Parameters**
* True if the function produces the same output as the reference, False otherwise

## ref_avgpool2d

The reference average pooling implementation from TensorFlow. 

**Input Parameters**
* X -- The input tensor
* pool_size -- The size of the pool (pool_rows, pool_cols)
* strides -- The strides (stride_rows, stride_cols)
* padding -- "valid" for no padding, "same" for zero padding

**Output Parameters**
* The result tensor of the convolution

## check_avgpool2d

Checks an input function with the given parameters against the reference implementation of TensorFlow. 

**Input Parameters**
* fn -- The function to compare to
* shape -- The shape to test the function with

**Output Parameters**
* True if the function produces the same output as the reference, False otherwise

In [92]:
def ref_maxpool2d(X, pool_size, strides, padding):
    return MaxPooling2D(pool_size=pool_size, strides=strides, padding=padding)(X).numpy()
    
def check_maxpool2d(fn, shape, pool_size, strides, padding):
    np.random.seed(3)
    shape = np.array(shape)
    
    X = create_random_input(shape)
    
    ref = ref_maxpool2d(X, pool_size, strides, padding)
    res = fn(X, pool_size, strides, padding)
    
    return (res == ref).all()
    
def ref_avgpool2d(X, pool_size, strides, padding):
    return AveragePooling2D(pool_size=pool_size, strides=strides, padding=padding)(X).numpy()
    
def check_avgpool2d(fn, shape, pool_size, strides, padding):
    np.random.seed(3)
    shape = np.array(shape)
    
    X = create_random_input(shape)
    
    ref = ref_avgpool2d(X, pool_size, strides, padding)
    res = fn(X, pool_size, strides, padding)
    
    return np.allclose(res, ref)
    return (res == ref).all()

# Pooling Layer Implementation

The actual maxpool2d and avgpool2d activation layer implementations. These function have the same input and output parameters as the ref_maxpool2d and ref_avgpool2d functions. 

In [93]:
def res_maxpool2d(X, pool_sizes, strides, padding):
    # extract parameters
    (batch, rows_in, cols_in, channels) = X.shape
    (pool_rows, pool_cols) = pool_sizes
    (stride_rows, stride_cols) = strides
    
    # same padding
    if padding == "same":    
        # calculate output dimensions
        rows_out = (rows_in + strides[0] - 1) // strides[0]
        cols_out = (cols_in + strides[1] - 1) // strides[1]
        
        # create output buffer
        out = np.zeros((batch, rows_out, cols_out, channels))
        out.fill(sys.float_info.min)

        # pool
        for i in range(batch):
            for y in range(rows_out):
                for x in range(cols_out):
                    sy = y * strides[0]
                    sx = x * strides[1]
                    for c in range(channels):
                        for ky in range(sy, sy + pool_rows):
                            for kx in range(sx, sx + pool_cols):
                                if ky >= 0 and ky < rows_in and kx >= 0 and kx < cols_in:
                                    out[i, y, x, c] = max(out[i, y, x, c], X[i, ky , kx, c])

        return out
    
    # zero padding
    else:
        rows_out = (rows_in - (pool_rows - strides[0])) // strides[0]
        cols_out = (cols_in - (pool_cols - strides[1])) // strides[1]
        
    
        # create output buffer
        out = np.zeros((batch, rows_out, cols_out, channels))

        # pool
        for i in range(batch):
            for y in range(rows_out):
                for x in range(cols_out):
                    sy = y * strides[0]
                    sx = x * strides[1]
                    for c in range(channels):
                        for ky in range(sy, sy + pool_rows):
                            for kx in range(sx, sx + pool_cols):
                                out[i, y, x, c] = max(out[i, y, x, c], X[i, ky , kx, c])

        return out
    
print(check_maxpool2d(res_maxpool2d, (2, 128, 128, 4), (2, 3), (4, 5), "same"))
print(check_maxpool2d(res_maxpool2d, (2, 128, 128, 4), (2, 3), (4, 5), "valid"))

def res_avgpool2d(X, pool_sizes, strides, padding):
    # extract parameters
    (batch, rows_in, cols_in, channels) = X.shape
    (pool_rows, pool_cols) = pool_sizes
    (stride_rows, stride_cols) = strides
    
    # same padding
    if padding == "same":    
        # calculate output dimensions
        rows_out = (rows_in + strides[0] - 1) // strides[0]
        cols_out = (cols_in + strides[1] - 1) // strides[1]
        
        # create output buffer
        out = np.zeros((batch, rows_out, cols_out, channels))

        # pool
        for i in range(batch):
            for y in range(rows_out):
                for x in range(cols_out):
                    sy = y * strides[0]
                    sx = x * strides[1]
                    for c in range(channels):
                        area = 0
                        for ky in range(sy, sy + pool_rows):
                            for kx in range(sx, sx + pool_cols):
                                if ky >= 0 and ky < rows_in and kx >= 0 and kx < cols_in:
                                    out[i, y, x, c] += X[i, ky , kx, c]
                                    area += 1
                        out[i, y, x, c] /= area

        return out
    
    # zero padding
    else:
        rows_out = (rows_in - (pool_rows - strides[0])) // strides[0]
        cols_out = (cols_in - (pool_cols - strides[1])) // strides[1]
        
    
        # create output buffer
        out = np.zeros((batch, rows_out, cols_out, channels))

        # pool
        for i in range(batch):
            for y in range(rows_out):
                for x in range(cols_out):
                    sy = y * strides[0]
                    sx = x * strides[1]
                    for c in range(channels):
                        for ky in range(sy, sy + pool_rows):
                            for kx in range(sx, sx + pool_cols):
                                out[i, y, x, c] += X[i, ky , kx, c]
                        out[i, y, x, c] /= pool_rows * pool_cols

        return out
 
print(check_avgpool2d(res_avgpool2d, (2, 128, 128, 4), (2, 3), (4, 5), "same"))
print(check_avgpool2d(res_avgpool2d, (2, 128, 128, 4), (2, 3), (4, 5), "valid"))

True
True
True
True
